In [1]:
import os

if os.getcwd().split('/')[-1] == 'evaluation':
    os.chdir('../')

## Which Hyperparameters Should I Iterate On?
https://huggingface.co/sentence-transformers/all-MiniLM-L12-v2

Here are typically the hyperparameters you should iterate on:

- model: the LLM to use for generation.
- prompt template: the variation of prompt templates to use for generation.
- temperature: the temperature value to use for generation.
- max tokens: the max token limit to set for your LLM generation.
- top-K: the number of retrieved nodes in your retrieval_context in a RAG pipeline.
- chunk size: the size of the retrieved nodes in your retrieval_context in a RAG pipeline.
- reranking model: the model used to rerank the retrieved nodes in your retrieval_context in a RAG pipeline.


In [15]:
import pandas as pd
from deepeval.test_case import LLMTestCase
from deepeval.dataset import EvaluationDataset

from deepeval import evaluate
from deepeval.metrics import AnswerRelevancyMetric

from chat_caller import query_gpt_chat, reload_vector_store
from deepeval.metrics import ContextualPrecisionMetric, FaithfulnessMetric, ContextualRecallMetric, ContextualRelevancyMetric, HallucinationMetric


In [3]:
df_questions = pd.read_csv('evaluation/questions.csv')

In [4]:
df_questions

,ques_id,context,question,answer,source_doc
0,Q1,- **AI Chatbot**: An AI chatbot is available f...,What is the purpose of the AI chatbot in the b...,The AI chatbot is designed to answer frequentl...,course_material/general/syllabus.md
1,Q2,Primer on Business Analytics with Python\nModu...,What are the basic techniques in text analytic...,The basic techniques in text analytics covered...,course_material/module 5/lecture slides.pptx
2,Q3,Primer on Business Analytics \nwith Python\nMo...,What is the business example of using linear r...,Predicting sales based on advertising spend.,course_material/module 2/lecture slides.pdf
3,Q4,Primer on Business Analytics with Python\nModu...,What are the three tasks in the online exam fo...,The three tasks in the online exam are Code In...,course_material/module 6/lecture slides.pptx
4,Q5,Primer on Business Analytics with Python\nModu...,What are some common algorithms used in superv...,Some common algorithms used in supervised mach...,course_material/module 4/lecture slides.pptx
5,Q6,"[Slide 1: Title Slide]\nHello again, everyone!...",What is the purpose of a t-test in business an...,A t-test allows us to compare the means of two...,course_material/module 2/transcript.txt
6,Q7,# Primer on Business Analytics with Python\n##...,What are the main concepts covered in the Busi...,The main concepts covered in the course are ba...,course_material/module 6/transcript.txt
7,Q8,"Thank you for your participation, and I'm exci...",What is the speaker excited to continue in the...,The speaker is excited to continue the learnin...,course_material/module 2/transcript.txt
8,Q9,"[Slide 1: Title Slide]\nHello, everyone! Welco...",What is the go-to tool for data manipulation a...,The go-to tool for data manipulation and analy...,course_material/module 1/transcript.txt
9,Q10,- Split data into training and test sets for u...,What is a confusion matrix in model evaluation?\n,A confusion matrix is a table that describes t...,course_material/module 4/lecture slides.pptx


In [5]:
df_evaluation = df_questions.copy()

In [6]:
import os
from huggingface_hub import login

# Retrieve the token from the environment
token = os.getenv("HUGGINGFACE_TOKEN")

if token:
    login(token=token)
    print("Logged in successfully.")
else:
    print("Environment token not found.")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /Users/crystalfrancis/.cache/huggingface/token
Login successful
Logged in successfully.


https://huggingface.co/spaces/mteb/leaderboard
- Model Types: Open, Sentence Transformers
- Model Sizes: <100M

In [7]:
embedding_models = [
    "sentence-transformers/all-MiniLM-L6-v2",  # Efficient and fast model for sentence embeddings
    'sentence-transformers/all-MiniLM-L12-v2',
    'TaylorAI/gte-tiny',
]

In [9]:
eval_metrics = {
    'Answer Relevance': AnswerRelevancyMetric(threshold=0.5), 
    'Faithfulness':FaithfulnessMetric(
        threshold=0.7,
        model="gpt-4",
        include_reason=True
    ),
    'Contextual Relevancy': ContextualRelevancyMetric(
        threshold=0.7,
        model="gpt-4",
        include_reason=True
    ),
    # The following metrics through errors regarding parsing json in the LLM response:
    #    'Contextual Precision': ContextualPrecisionMetric(
    #         threshold=0.7,
    #         model="gpt-4",
    #         include_reason=True
    #     ),
    #     'Hallucination': HallucinationMetric(threshold=0.5)
    # 'Contextual Recall': ContextualRecallMetric(
    #     threshold=0.7,
    #     model="gpt-4",
    #     include_reason=True
    # ),
}

In [10]:
def add_metrics(dataset, model, df_model):
    df_model['Model'] = model
    results = evaluate(dataset, list(eval_metrics.values()))

    metric_labels = list(eval_metrics.keys())

    for m_index in range(len(metric_labels)):
        metric_label = metric_labels[m_index]
        df_model[metric_label] = [
            r.metrics_metadata[m_index].score for r in results
        ]
        df_model[f"{metric_label} - Reason"] = [ 
            r.metrics_metadata[m_index].reason for r in results
        ]

    return df_model


In [11]:
def evaluate_rag(model, df_questions):
    print("Creating Test Cases...")
    print("--------------------------------------------------------------------------------")
    reload_vector_store(model)
    df_model = df_questions.copy()
    test_cases = []

    for _, row in df_model.iterrows():
        question = row['question']
        answer = query_gpt_chat(question, [])[1]
        test = LLMTestCase(
            input=question,
            actual_output=answer,
            expected_output=row['answer'],
            retrieval_context=[row['context'] or '']
        )
        test_cases.append(test)
        
    print("Evaluating Test Cases...")
    print("--------------------------------------------------------------------------------")
    dataset = EvaluationDataset(test_cases=test_cases)
    df_model = add_metrics(dataset, model, df_model)

    print("--------------------------------------------------------------------------------")
    print("Completed Evaluation")
    return df_model


In [12]:
df_results = None
for model in embedding_models:
    df_model = evaluate_rag(model, df_questions)
    df_results = df_model if df_results is None else pd.concat([df_results, df_model])

Creating Test Cases...
--------------------------------------------------------------------------------
load INSTRUCTOR_Transformer
max_seq_length  512


I0000 00:00:1723202705.841102 28273155 work_stealing_thread_pool.cc:320] WorkStealingThreadPoolImpl::PrepareFork
I0000 00:00:1723202708.012568 28273155 work_stealing_thread_pool.cc:320] WorkStealingThreadPoolImpl::PrepareFork


Prompt tokens: 599
Completion tokens: 84
Total tokens: 683
Prompt tokens: 577
Completion tokens: 176
Total tokens: 753
Prompt tokens: 597
Completion tokens: 130
Total tokens: 727
Prompt tokens: 605
Completion tokens: 100
Total tokens: 705
Prompt tokens: 528
Completion tokens: 146
Total tokens: 674
Prompt tokens: 568
Completion tokens: 118
Total tokens: 686
Prompt tokens: 588
Completion tokens: 187
Total tokens: 775
Prompt tokens: 492
Completion tokens: 80
Total tokens: 572
Prompt tokens: 634
Completion tokens: 78
Total tokens: 712


Output()

Prompt tokens: 582
Completion tokens: 180
Total tokens: 762
Evaluating Test Cases...
--------------------------------------------------------------------------------
Evaluating test cases...
Event loop is already running. Applying nest_asyncio patch to allow async execution...


...

I0000 00:00:1723203432.394619 28273155 work_stealing_thread_pool.cc:320] WorkStealingThreadPoolImpl::PrepareFork


I0000 00:00:1723203440.346830 28273155 work_stealing_thread_pool.cc:320] WorkStealingThreadPoolImpl::PrepareFork




Metrics Summary

  - ✅ Answer Relevancy (score: 0.6666666666666666, threshold: 0.5, strict: False, evaluation model: gpt-4o, reason: The score is 0.67 because the output provides relevant information about the AI chatbot's purpose in the business analytics course, but includes an irrelevant statement about the syllabus., error: None)
  - ✅ Faithfulness (score: 1.0, threshold: 0.7, strict: False, evaluation model: gpt-4, reason: The score is 1.00 because there are no contradictions between the retrieval context and the actual output., error: None)
  - ✅ Contextual Relevancy (score: 1.0, threshold: 0.7, strict: False, evaluation model: gpt-4, reason: The score is 1.00 because the retrieval context perfectly matches the input, indicating high relevancy and accuracy., error: None)

For test case:

  - input: What is the purpose of the AI chatbot in the business analytics course?

  - actual output: The AI chatbot in the business analytics course is designed to answer frequently asked que

✅ Tests finished! Run "deepeval login" to view evaluation results on the web.

--------------------------------------------------------------------------------
Completed Evaluation


In [13]:
df_results

,ques_id,context,question,answer,source_doc,Model,Answer Relevance,Answer Relevance - Reason,Faithfulness,Faithfulness - Reason,Contextual Relevancy,Contextual Relevancy - Reason
0,Q1,- **AI Chatbot**: An AI chatbot is available f...,What is the purpose of the AI chatbot in the b...,The AI chatbot is designed to answer frequentl...,course_material/general/syllabus.md,sentence-transformers/all-MiniLM-L6-v2,0.750000,The score is 0.75 because the answer mostly ad...,1.000000,The score is 1.00 because there are no contrad...,1.0,The score is 1.00 because there were no reason...
1,Q2,Primer on Business Analytics with Python\nModu...,What are the basic techniques in text analytic...,The basic techniques in text analytics covered...,course_material/module 5/lecture slides.pptx,sentence-transformers/all-MiniLM-L6-v2,1.000000,The score is 1.00 because the output is fully ...,0.866667,The score is 0.87 because the actual output in...,1.0,The score is 1.00 because the retrieval contex...
2,Q3,Primer on Business Analytics \nwith Python\nMo...,What is the business example of using linear r...,Predicting sales based on advertising spend.,course_material/module 2/lecture slides.pdf,sentence-transformers/all-MiniLM-L6-v2,0.833333,The score is 0.83 because the majority of the ...,1.000000,The score is 1.00 because there are no contrad...,1.0,The score is 1.00 because the retrieval contex...
3,Q4,Primer on Business Analytics with Python\nModu...,What are the three tasks in the online exam fo...,The three tasks in the online exam are Code In...,course_material/module 6/lecture slides.pptx,sentence-transformers/all-MiniLM-L6-v2,0.875000,The score is 0.88 because the answer effective...,1.000000,The score is 1.00 because there were no contra...,1.0,The score is 1.00 because the retrieval contex...
4,Q5,Primer on Business Analytics with Python\nModu...,What are some common algorithms used in superv...,Some common algorithms used in supervised mach...,course_material/module 4/lecture slides.pptx,sentence-transformers/all-MiniLM-L6-v2,0.916667,The score is 0.92 because the answer is highly...,1.000000,The score is 1.00 because there are no contrad...,1.0,The score is 1.00 because the retrieval contex...
5,Q6,"[Slide 1: Title Slide]\nHello again, everyone!...",What is the purpose of a t-test in business an...,A t-test allows us to compare the means of two...,course_material/module 2/transcript.txt,sentence-transformers/all-MiniLM-L6-v2,0.800000,The score is 0.80 because the explanation abou...,1.000000,The score is 1.00 because there are no contrad...,1.0,The score is 1.00 because the retrieval contex...
6,Q7,# Primer on Business Analytics with Python\n##...,What are the main concepts covered in the Busi...,The main concepts covered in the course are ba...,course_material/module 6/transcript.txt,sentence-transformers/all-MiniLM-L6-v2,0.947368,The score is 0.95 because the answer is highly...,0.823529,The score is 0.82 because the actual output in...,1.0,The score is 1.00 because there are no reasons...
7,Q8,"Thank you for your participation, and I'm exci...",What is the speaker excited to continue in the...,The speaker is excited to continue the learnin...,course_material/module 2/transcript.txt,sentence-transformers/all-MiniLM-L6-v2,0.250000,The score is 0.25 because the response include...,1.000000,The score is 1.00 because there are no contrad...,1.0,The score is 1.00 because the retrieval contex...
8,Q9,"[Slide 1: Title Slide]\nHello, everyone! Welco...",What is the go-to tool for data manipulation a...,The go-to tool for data manipulation and analy...,course_material/module 1/transcript.txt,sentence-transformers/all-MiniLM-L6-v2,0.750000,The score is 0.75 because the main response co...,1.000000,The score is 1.00 because there are no contrad...,1.0,The score is 1.00 because the retrieval contex...
9,Q10,- Split data into training and test sets for u...,What is a confusion matrix in model evaluation?\n,A confusion matrix is a table that describes 

In [14]:
df_results.to_csv('evaluation/evaluations_v4.csv', index=False)